In [1]:
import os

import arcpy

import numpy

import tidegates

workspace = r"..\MB_InputLayers.gdb"

SEALEVELRISE = numpy.arange(0, 7) # 0 through 6 in 1-ft increments

In [2]:
for surge in tidegates.SURGES.keys():
    for slr in SEALEVELRISE:
        layer_name = "prelim_flood_surge{}_rise{}".format(surge, slr)
        print(layer_name)
        with tidegates.utils.WorkSpace(workspace), tidegates.utils.OverwriteState(True):
            results = tidegates.flood_area(
                dem="cape_dem", 
                polygons="ZOI", 
                tidegate_column="GeoID",
                storm_surge=surge,
                sea_level_rise=slr, 
                filename=layer_name,
                workspace=workspace
            )
            
            arcpy.management.AddField(
                in_table=results.getOutput(0),
                field_name='SLR',
                field_type='DOUBLE'
            )
            
            arcpy.management.AddField(
                in_table=results.getOutput(0),
                field_name='surge',
                field_type='TEXT',
                field_length=10
            )
            
            cur = arcpy.UpdateCursor(results.getOutput(0))
            for row in cur:
                row.SLR = slr
                row.surge = surge
                cur.updateRow(row)

            del cur, row
            
#         arcpy.management.CopyFeatures(
#             in_features='{}/{}'.format(ws, layer_name),
#             out_feature_class="{}.shp".format(layer_name)
#         )

prelim_flood_surge10-yr_rise0
1/9 ..\MB_InputLayers.gdb
2/9 cape_dem raster loaded
3/9 ZOI polygon processed
4/9 topo clipped
5/9 rasters to arrays
6/9 mask things
7/9 coverted back to raster and saved
8/9 convert to polygon in ..\MB_InputLayers.gdb


ExecuteError: ERROR 000582: Error occurred during execution.


In [ ]:
debug

In [ ]:
layers = []
for slr in range(7):
    for surge in SURGES.keys():
        layers.append("prelim_flood_surge{}_rise{}".format(surge, slr))

with tidegates.utils.WorkSpace(ws), tidegates.utils.OverwriteState(True):
    arcpy.management.Merge(layers, 'merged_01')

In [ ]:
if 1:
    surge = '10yr'
    for slr in SEALEVELRISE:
        elevation = MHHW + SURGES[surge] + slr * 0.5
        layer_name = "prelim_flood_surge{}_rise{}".format(surge, n)
        print(layer_name)
        with tidegates.utils.WorkSpace(ws), tidegates.utils.OverwriteState(True):
            tidegates.flood_area(
                dem="cape_dem", 
                polygons="ZOI", 
                tidegate_column="GeoID", 
                elevation=elevation, 
                filename=layer_name
            )

In [ ]:
if 1:
    surge = '50yr'
    for slr in SEALEVELRISE:
        elevation = MHHW + SURGES[surge] + slr * 0.5
        layer_name = "prelim_flood_surge{}_rise{}".format(surge, slr)
        print(layer_name)
        with tidegates.utils.WorkSpace(ws), tidegates.utils.OverwriteState(True):
            tidegates.flood_area(
                dem="cape_dem", 
                polygons="ZOI", 
                tidegate_column="GeoID", 
                elevation=elevation, 
                filename=layer_name
            )

In [ ]:
pwd

In [ ]:
# check the max/min elevations of each ZOI
with tidegates.utils.Extension("spatial"), tidegates.utils.WorkSpace(ws):
    arcpy.sa.ZonalStatisticsAsTable(
        in_zone_data="ZOI", 
        zone_field="GeoID",
        in_value_raster="cape_dem",
        out_table="max_elev",
    )
    

In [ ]:
import fiona

In [ ]:
def compileFloods(template, outputshp, gate_col="gridcode", mode='w'):

    #_check_mode(mode)
    with fiona.open(template, 'r') as src:
        shp_params = dict(driver=src.driver, crs=src.crs, schema=src.schema)

    with fiona.open(outputshp, mode, **shp_params) as outshp:
        for slr in SEALEVELRISE:
            for surge in SURGES.keys():
                layer_name = "prelim_flood_surge{}_rise{}".format(surge, slr)
                dummy = arcpy.management.CopyFeatures(
                    in_features=layer_name,
                    out_feature_class='../dummy.shp'

                )

                with fiona.open(dummy.getOutput(0)) as newshp:
                    for record in newshp:
                        record['properties']['SLR'] = slr
                        record['properties']['surge'] = surge

                        if record['properties'][gate_col] > 0:
                            outshp.write(record)


In [ ]:
with tidegates.utils.WorkSpace(ws), tidegates.utils.OverwriteState(True):
    compileFloods("../flood_template.shp", "../mb_floods.shp")

In [ ]:
debug